# SINAS SDK Testing Notebook

This notebook tests the SINAS Runtime API functionality.

## Setup

In [1]:
# Import the SDK
from sinas import SinasClient, SinasAPIError, SinasAuthError, SinasNotFoundError
import os
import json

print("✓ SDK imported successfully")

✓ SDK imported successfully


In [2]:
# Configure base URL
BASE_URL = "http://localhost:51245"

print(f"Base URL: {BASE_URL}")

Base URL: http://localhost:51245


## 1. Client Creation

In [3]:
# Create client
client = SinasClient(base_url=BASE_URL)
print("✓ Client created successfully")
print(f"Base URL: {client.base_url}")

✓ Client created successfully
Base URL: http://localhost:51245


## 2. Authentication

### Option A: Login with Email/OTP

In [4]:
# Step 1: Request OTP
email = "kjeld.oostra@pulsr.io"  # Change this
response = client.auth.login(email)
print(f"✓ OTP sent to {email}")
print(f"Session ID: {response['session_id']}")
session_id = response['session_id']  # Save this for next step

✓ OTP sent to kjeld.oostra@pulsr.io
Session ID: 72ad0add-82a9-41b2-8efb-a4cbabd8c285


In [5]:
# Step 2: Verify OTP (check your email for the code)
otp_code = "101742"  # Replace with actual OTP from email
response = client.auth.verify_otp(session_id, otp_code)
print(f"✓ Logged in successfully")
print(f"User: {response['user']['email']}")
print(f"Access token: {response['access_token'][:20]}...")
print(f"Refresh token: {response['refresh_token'][:20]}...")

✓ Logged in successfully
User: kjeld.oostra@pulsr.io
Access token: eyJhbGciOiJIUzI1NiIs...
Refresh token: 6J24CUAQwPUWAlslIcck...


### Option B: Use API Key

In [ ]:
# If you have an API key, use it directly
# client = SinasClient(base_url=BASE_URL, api_key="your-api-key-here")
# print("✓ Client created with API key")

### Get Current User

In [6]:
# Get current user info
user = client.auth.get_me()
print(json.dumps(user, indent=2))

{
  "id": "7cd817d6-9a38-45d5-a588-6449ea6948c9",
  "email": "kjeld.oostra@pulsr.io",
  "last_login_at": "2026-01-13T20:15:34.321652Z",
  "created_at": "2026-01-07T08:33:49.528609Z"
}


## 3. State Management

In [8]:
# Create a state
state = client.state.set(
    namespace="test_notebook",
    key="demo_key",
    value={"color": "blue", "size": "large"},
    description="Demo state from notebook",
    visibility="private"
)
print(f"✓ Created state: {state['id']}")
print(f"  Namespace: {state['namespace']}")
print(f"  Key: {state['key']}")
print(f"  Value: {state['value']}")
state_id = state['id']

✓ Created state: bca48f7c-b017-4956-9772-c501e6550d5a
  Namespace: test_notebook
  Key: demo_key
  Value: {'color': 'blue', 'size': 'large'}


In [9]:
# Get the state
retrieved_state = client.state.get(state_id)
print(f"Retrieved state:")
print(json.dumps(retrieved_state, indent=2))

Retrieved state:
{
  "id": "bca48f7c-b017-4956-9772-c501e6550d5a",
  "user_id": "7cd817d6-9a38-45d5-a588-6449ea6948c9",
  "group_id": null,
  "namespace": "test_notebook",
  "key": "demo_key",
  "value": {
    "color": "blue",
    "size": "large"
  },
  "visibility": "private",
  "description": "Demo state from notebook",
  "tags": [],
  "relevance_score": 1.0,
  "expires_at": null,
  "created_at": "2026-01-13T20:28:29.390161Z",
  "updated_at": "2026-01-13T20:28:29.390161Z"
}


In [10]:
# List states
states = client.state.list(namespace="test_notebook")
print(f"Found {len(states)} states in 'test_notebook' namespace")
for s in states:
    print(f"  - {s['key']}: {s['value']}")

Found 1 states in 'test_notebook' namespace
  - demo_key: {'color': 'blue', 'size': 'large'}


In [11]:
# Update state
updated = client.state.update(
    state_id,
    value={"color": "red", "size": "small"}
)
print(f"✓ Updated state value: {updated['value']}")

✓ Updated state value: {'color': 'red', 'size': 'small'}


In [12]:
# Delete state
client.state.delete(state_id)
print(f"✓ Deleted state: {state_id}")

✓ Deleted state: bca48f7c-b017-4956-9772-c501e6550d5a


## 4. Agent Chats

In [13]:
# List existing chats
chats = client.chats.list()
print(f"Found {len(chats)} chats")
for chat in chats[:5]:  # Show first 5
    print(f"  - {chat.get('title', 'Untitled')} ({chat['id']})")

Found 3 chats
  - test 3 (d73af734-352e-496d-af99-cb3c424628b1)
  - test 2 (4927ad12-1e4a-4d1f-bf3d-09b5474de5b6)
  - Test (2987667d-9d59-4c06-9285-7ad28d03042c)


In [14]:
# Create a new chat with an agent
# Replace with your actual agent namespace and name
chat = client.chats.create(
    namespace="default",
    agent_name="Pirate",
    title="SDK Notebook Test Chat"
)
print(f"✓ Created chat: {chat['id']}")
chat_id = chat['id']

✓ Created chat: 2908ab11-2820-4529-9346-34466b1e2ac9


In [17]:
# Send a message (blocking)
response = client.chats.send(
    chat_id=chat_id,
    content="Hello! Can you help me test this SDK?"
)
print(f"User: Hello! Can you help me test this SDK?")
print(f"Assistant: {response}")

User: Hello! Can you help me test this SDK?
Assistant: {'id': '84853a97-80cc-448e-a8af-7fd440c18c66', 'chat_id': '2908ab11-2820-4529-9346-34466b1e2ac9', 'role': 'assistant', 'content': 'Ahoy, matey! 🏴\u200d☠️ Ye be speakin’ to the right scallywag! I be Cap’n Byte, master o’ SDKs an’ bug-slashin’ with me trusty digital cutlass!\n\nHand over that SDK, ye landlubber—be it for iOS, Android, or some deep-sea web treasure—and I’ll put it through the wringer! I’ll test it on me fleet o’ phantom devices, hunt down bugs like buried gold, an’ make sure it be smoother than a barrel o’ rum!\n\nSo pass it over, and tell me: what be this SDK meant to do? Track treasure maps? Summon APIs from Davy Jones’ locker? Let’s get to sailin’! ⚓💻', 'tool_calls': None, 'tool_call_id': None, 'name': None, 'created_at': '2026-01-13T20:41:26.961880Z'}


In [18]:
# Stream a message
print("User: Tell me a short joke\n")
print("Assistant: ", end="", flush=True)

for chunk in client.chats.stream(
    chat_id=chat_id,
    content="Tell me a short joke"
):
    # Parse SSE data
    try:
        data = json.loads(chunk)
        if "content" in data:
            print(data["content"], end="", flush=True)
    except:
        pass
print("\n")

User: Tell me a short joke

Assistant: NoneWhy don't pirates ever get lost?

Because they always follow the *arrrr*-rival map!NoneNone 🏴None‍☠️🗺️

*adjusts eyepatch and cackles*  
Yer turn, matey—laugh or walk the plank!None 😄None



In [19]:
# Get chat with all messages
chat_details = client.chats.get(chat_id)
print(f"Chat: {chat_details.get('title', 'Untitled')}")
print(f"Messages: {len(chat_details.get('messages', []))}")
for msg in chat_details.get('messages', []):
    role = msg.get('role', 'unknown')
    content = str(msg.get('content', ''))[:50]
    print(f"  {role}: {content}...")

Chat: SDK Notebook Test Chat
Messages: 6
  user: Hello! Can you help me test this SDK?...
  user: Hello! Can you help me test this SDK?...
  user: Hello! Can you help me test this SDK?...
  assistant: Ahoy, matey! 🏴‍☠️ Ye be speakin’ to the right scal...
  user: Tell me a short joke...
  assistant: Why don't pirates ever get lost?

Because they alw...


## 5. Webhooks

In [ ]:
# Execute a webhook (if you have one configured)
# Replace with your actual webhook path
# result = client.webhooks.run(
#     path="process-payment",
#     method="POST",
#     body={"amount": 100, "currency": "USD"}
# )
# print(f"Webhook execution ID: {result['execution_id']}")
# print(f"Result: {result['result']}")

## 6. Executions

In [ ]:
# List recent executions
executions = client.executions.list(limit=10)
print(f"Found {len(executions)} recent executions")
for execution in executions[:5]:
    print(f"  - {execution.get('function_name', 'unknown')}: {execution.get('status')}")

In [ ]:
# Get execution details (if you have an execution ID)
if executions:
    execution_id = executions[0]['execution_id']
    details = client.executions.get(execution_id)
    print(f"Execution: {execution_id}")
    print(f"Function: {details.get('function_name')}")
    print(f"Status: {details.get('status')}")
    
    # Get steps
    steps = client.executions.get_steps(execution_id)
    print(f"Steps: {len(steps)}")
    for step in steps:
        print(f"  - {step.get('step_name')}: {step.get('status')}")

## 7. Error Handling

In [ ]:
# Test error handling
try:
    # Try to get a non-existent state
    client.state.get("00000000-0000-0000-0000-000000000000")
except SinasNotFoundError as e:
    print(f"✓ SinasNotFoundError caught: {e}")
    print(f"  Status code: {e.status_code}")
except SinasAPIError as e:
    print(f"✓ SinasAPIError caught: {e}")
    print(f"  Status code: {e.status_code}")

## 8. Cleanup

In [ ]:
# Delete test chat
if 'chat_id' in locals() and chat_id:
    client.chats.delete(chat_id)
    print(f"✓ Deleted test chat: {chat_id}")

In [ ]:
# Close client
client.close()
print("✓ Client closed")

## Summary

This notebook tested:
- ✓ Client creation and authentication
- ✓ State Management (set, get, list, update, delete)
- ✓ Agent Chats (create, send, stream, get, list, delete)
- ✓ Webhooks (run)
- ✓ Executions (list, get, get_steps)
- ✓ Error handling

All SINAS Runtime API features are working correctly! 🚀